<a href="https://colab.research.google.com/github/Mohammed-Taasir/IR_2023-GROUP-14/blob/assignment-1/IR_Assignment_1_Question_1_and_2_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp '/content/drive/MyDrive/IR Assignment-1/CSE508_Winter2023_Dataset.zip' '/content/'
!unzip 'CSE508_Winter2023_Dataset.zip' &> /dev/null
!rm 'CSE508_Winter2023_Dataset.zip'

In [3]:
path = '/content/CSE508_Winter2023_Dataset/'

In [4]:
import glob
import pandas as pd
import numpy as np
import string
import os
import re
import pickle
import nltk

In [5]:
from sortedcontainers import SortedDict, SortedList, SortedSet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import PlaintextCorpusReader 
from nltk.stem.wordnet import WordNetLemmatizer

In [6]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# * Defining Helper Functions

### Read from files

In [7]:
def getListOfFiles(directory):

    # Parameters: directory: type(string)        
    # returns: list of all files in directory with the full path of file
    
    list_of_files = []

    for i in range(len(filenames)):
      fi = "/content/CSE508_Winter2023_Dataset/"+filenames[i]
      list_of_files.append(fi)
    
    return list_of_files

### Preprocessing Functions

In [8]:
def lowercase(data):

    # Parameters: data: type(string)
    # returns: lowercase of data   
     
    return data.lower()

In [9]:
def perform_word_tokenize(corpus):
  
    # Parameters:corpus: type(string)   
    # returns word-level tokenization of corpus

    return word_tokenize(corpus)

In [10]:
def remove_stopwords_from_tokens(tokens, stopwords_set):
  
    # Parameters: tokens: type(list)
    #             stopwords_set: type(set)
    # returns: tokens without stopwords

    tokens_sans_stopwords = [x for x in tokens if x not in stopwords_set] 
    return tokens_sans_stopwords

In [11]:
def remove_punctuation_from_tokens(tokens):

    # Parameters: tokens: type(list)
    # returns: tokens without punctuation

    tokens_sans_punctuation = [x.translate(str.maketrans('', '', string.punctuation)) for x in tokens]
    return tokens_sans_punctuation

In [12]:
def remove_blank_space_tokens(tokens):
    
    #Parameters: tokens: type(list)
    #returns: tokens without blank tokens

    tokens_sans_blank_space = [x for x in tokens if x!='']  
    return tokens_sans_blank_space

In [13]:
def preprocess(corpus, stopwords_set, preprocess_type):
    # Convert the text to lower case
    lowercase_corpus = lowercase(corpus)
    
    # Perform word tokenization (word_tokenize also takes care of whitespace)
    word_tokens = perform_word_tokenize(lowercase_corpus)
    
    # Remove stopwords from tokens
    word_tokens_sans_stopwords = remove_stopwords_from_tokens(word_tokens, stopwords_set)
    
    # Remove punctuation marks from tokens
    word_tokens_sans_punctuation = remove_punctuation_from_tokens(word_tokens_sans_stopwords)
    
    # Remove blank space tokens
    word_tokens_sans_blank_tokens = remove_blank_space_tokens(word_tokens_sans_punctuation)
  
    
    if preprocess_type=='query':
        return word_tokens_sans_blank_tokens                              # if its a query return tokens in the same order
    return sorted(list(dict.fromkeys(word_tokens_sans_blank_tokens)))     # if its a whole document then return tokens in sorted manner

In [14]:
def create_file_dictionary(list_of_files):
    
    # Paramteres: list_of_files: type(string)
    # returns: file_dictionary with integer key and path_of_file as value
  
    file_dictionary = {}
    for i in range(len(list_of_files)):
        file_dictionary[i] = list_of_files[i]
    
    return file_dictionary

# Q1. Data Preprocessing

## (i) Relevant Text Extraction

In [15]:
corpus_root='/content/CSE508_Winter2023_Dataset/'
corpus=PlaintextCorpusReader(corpus_root,'.*')

In [16]:
# Printing names of all files

filenames=corpus.fileids()
print(filenames)

['cranfield0001', 'cranfield0002', 'cranfield0003', 'cranfield0004', 'cranfield0005', 'cranfield0006', 'cranfield0007', 'cranfield0008', 'cranfield0009', 'cranfield0010', 'cranfield0011', 'cranfield0012', 'cranfield0013', 'cranfield0014', 'cranfield0015', 'cranfield0016', 'cranfield0017', 'cranfield0018', 'cranfield0019', 'cranfield0020', 'cranfield0021', 'cranfield0022', 'cranfield0023', 'cranfield0024', 'cranfield0025', 'cranfield0026', 'cranfield0027', 'cranfield0028', 'cranfield0029', 'cranfield0030', 'cranfield0031', 'cranfield0032', 'cranfield0033', 'cranfield0034', 'cranfield0035', 'cranfield0036', 'cranfield0037', 'cranfield0038', 'cranfield0039', 'cranfield0040', 'cranfield0041', 'cranfield0042', 'cranfield0043', 'cranfield0044', 'cranfield0045', 'cranfield0046', 'cranfield0047', 'cranfield0048', 'cranfield0049', 'cranfield0050', 'cranfield0051', 'cranfield0052', 'cranfield0053', 'cranfield0054', 'cranfield0055', 'cranfield0056', 'cranfield0057', 'cranfield0058', 'cranfield005

In [17]:
print(type(filenames))
print(len(filenames))

<class 'list'>
1400


## First 5 Documents

In [18]:
# Printing contents of first 5 files before extracting

for i in range(0, 5):
  outfile = open('/content/CSE508_Winter2023_Dataset/'+filenames[i], 'r')
  doc = outfile.read()
  print("------------")
  print("Document: "+str(i+1))
  print("------------")
  print(doc)
  print('-----------------------------------------------------------------')
  print('')
  outfile.close()

------------
Document: 1
------------
<DOC>
<DOCNO>
1
</DOCNO>
<TITLE>
experimental investigation of the aerodynamics of a
wing in a slipstream .
</TITLE>
<AUTHOR>
brenckman,m.
</AUTHOR>
<BIBLIO>
j. ae. scs. 25, 1958, 324.
</BIBLIO>
<TEXT>
  an experimental study of a wing in a propeller slipstream was
made in order to determine the spanwise distribution of the lift
increase due to slipstream at different angles of attack of the wing
and at different free stream to slipstream velocity ratios .  the
results were intended in part as an evaluation basis for different
theoretical treatments of this problem .
  the comparative span loading curves, together with supporting
evidence, showed that a substantial part of the lift increment
produced by the slipstream was due to a /destalling/ or boundary-layer-control
effect .  the integrated remaining lift increment,
after subtracting this destalling lift, was found to agree
well with a potential flow theory .
  an empirical evaluation of the des

## Extracting content between TITLE and TEXT tag from all documents.

In [19]:
for i in range(len(filenames)):
    outfile = open('/content/CSE508_Winter2023_Dataset/'+filenames[i], 'r')
    doc = outfile.read()

    # initializing string
    test_str = doc

    # initializing tags
    tag1 = "TITLE"
    tag2 = "TEXT"

    # regex to extract required strings
    reg_str1 = "<"+tag1+">(.*?)</"+tag1+">"
    res1 = re.findall(reg_str1, test_str, re.DOTALL)

    reg_str2 = "<"+tag2+">(.*?)</"+tag2+">"
    res2 = re.findall(reg_str2, test_str, re.DOTALL)

    #Combining contents of TITLE and TEXT
    res = res1+res2
    
    s = res

    # using list comprehension
    listToStr = ' '.join([str(elem) for elem in s])

    writeFile = open('/content/CSE508_Winter2023_Dataset/'+filenames[i], 'w')
    L = listToStr

    writeFile.write(L)
    writeFile.close()

    outfile.close()

## First 5 Documents after extracting contents between TITLE and TEXT tag.

In [20]:
for i in range(0, 5):
  outfile = open('/content/CSE508_Winter2023_Dataset/'+filenames[i], 'r')
  doc = outfile.read()
  print("------------")
  print("Document: "+str(i+1))
  print("------------")
  print(doc)
  print('-----------------------------------------------------------------')
  print('')
  outfile.close()

------------
Document: 1
------------

experimental investigation of the aerodynamics of a
wing in a slipstream .
 
  an experimental study of a wing in a propeller slipstream was
made in order to determine the spanwise distribution of the lift
increase due to slipstream at different angles of attack of the wing
and at different free stream to slipstream velocity ratios .  the
results were intended in part as an evaluation basis for different
theoretical treatments of this problem .
  the comparative span loading curves, together with supporting
evidence, showed that a substantial part of the lift increment
produced by the slipstream was due to a /destalling/ or boundary-layer-control
effect .  the integrated remaining lift increment,
after subtracting this destalling lift, was found to agree
well with a potential flow theory .
  an empirical evaluation of the destalling effects was made for
the specific configuration of the experiment .

-----------------------------------------------

# (ii) Preprocessing

## 1. Lowercase the text

In [21]:
newfiles=[]
for i in range(len(filenames)):
  fileind=open('/content/CSE508_Winter2023_Dataset/'+filenames[i], 'r')
  filedata=fileind.read()
  filedata=lowercase(filedata)
  newfiles.append(filedata)
print(len(newfiles))

for i in range(0, 5):
  print('----------')
  print('File : '+str(i+1))
  print('----------')
  print(newfiles[i])
  print('--------------------------------------------------------------------')

1400
----------
File : 1
----------

experimental investigation of the aerodynamics of a
wing in a slipstream .
 
  an experimental study of a wing in a propeller slipstream was
made in order to determine the spanwise distribution of the lift
increase due to slipstream at different angles of attack of the wing
and at different free stream to slipstream velocity ratios .  the
results were intended in part as an evaluation basis for different
theoretical treatments of this problem .
  the comparative span loading curves, together with supporting
evidence, showed that a substantial part of the lift increment
produced by the slipstream was due to a /destalling/ or boundary-layer-control
effect .  the integrated remaining lift increment,
after subtracting this destalling lift, was found to agree
well with a potential flow theory .
  an empirical evaluation of the destalling effects was made for
the specific configuration of the experiment .

-------------------------------------------------

## 2. Perform tokenization

In [22]:
for i in range(len(newfiles)):
  filedata=newfiles[i]
  filedata=perform_word_tokenize(filedata)
  newfiles[i] = filedata
print(len(newfiles))

for i in range(0, 5):
  print('----------')
  print('File : '+str(i+1))
  print('----------')
  print(newfiles[i])
  print('--------------------------------------------------------------------')

1400
----------
File : 1
----------
['experimental', 'investigation', 'of', 'the', 'aerodynamics', 'of', 'a', 'wing', 'in', 'a', 'slipstream', '.', 'an', 'experimental', 'study', 'of', 'a', 'wing', 'in', 'a', 'propeller', 'slipstream', 'was', 'made', 'in', 'order', 'to', 'determine', 'the', 'spanwise', 'distribution', 'of', 'the', 'lift', 'increase', 'due', 'to', 'slipstream', 'at', 'different', 'angles', 'of', 'attack', 'of', 'the', 'wing', 'and', 'at', 'different', 'free', 'stream', 'to', 'slipstream', 'velocity', 'ratios', '.', 'the', 'results', 'were', 'intended', 'in', 'part', 'as', 'an', 'evaluation', 'basis', 'for', 'different', 'theoretical', 'treatments', 'of', 'this', 'problem', '.', 'the', 'comparative', 'span', 'loading', 'curves', ',', 'together', 'with', 'supporting', 'evidence', ',', 'showed', 'that', 'a', 'substantial', 'part', 'of', 'the', 'lift', 'increment', 'produced', 'by', 'the', 'slipstream', 'was', 'due', 'to', 'a', '/destalling/', 'or', 'boundary-layer-control'

In [23]:
stopwords_set = set(stopwords.words('english'))

## 3. Remove stopwords

In [24]:
for i in range(len(newfiles)):
  filedata=newfiles[i]
  tokens_sans_stopwords = remove_stopwords_from_tokens(filedata, stopwords_set)
  newfiles[i] = tokens_sans_stopwords
print(len(newfiles))

for i in range(0, 5):
  print('----------')
  print('File : '+str(i+1))
  print('----------')
  print(newfiles[i])
  print('--------------------------------------------------------------------')

1400
----------
File : 1
----------
['experimental', 'investigation', 'aerodynamics', 'wing', 'slipstream', '.', 'experimental', 'study', 'wing', 'propeller', 'slipstream', 'made', 'order', 'determine', 'spanwise', 'distribution', 'lift', 'increase', 'due', 'slipstream', 'different', 'angles', 'attack', 'wing', 'different', 'free', 'stream', 'slipstream', 'velocity', 'ratios', '.', 'results', 'intended', 'part', 'evaluation', 'basis', 'different', 'theoretical', 'treatments', 'problem', '.', 'comparative', 'span', 'loading', 'curves', ',', 'together', 'supporting', 'evidence', ',', 'showed', 'substantial', 'part', 'lift', 'increment', 'produced', 'slipstream', 'due', '/destalling/', 'boundary-layer-control', 'effect', '.', 'integrated', 'remaining', 'lift', 'increment', ',', 'subtracting', 'destalling', 'lift', ',', 'found', 'agree', 'well', 'potential', 'flow', 'theory', '.', 'empirical', 'evaluation', 'destalling', 'effects', 'made', 'specific', 'configuration', 'experiment', '.']
--

## 4. Remove punctuations

In [25]:
for i in range(len(newfiles)):
  filedata=newfiles[i]
  tokens_sans_punctuation = remove_punctuation_from_tokens(filedata)
  newfiles[i] = tokens_sans_punctuation
print(len(newfiles))

for i in range(0, 5):
  print('----------')
  print('File : '+str(i+1))
  print('----------')
  print(newfiles[i])
  print('--------------------------------------------------------------------')

1400
----------
File : 1
----------
['experimental', 'investigation', 'aerodynamics', 'wing', 'slipstream', '', 'experimental', 'study', 'wing', 'propeller', 'slipstream', 'made', 'order', 'determine', 'spanwise', 'distribution', 'lift', 'increase', 'due', 'slipstream', 'different', 'angles', 'attack', 'wing', 'different', 'free', 'stream', 'slipstream', 'velocity', 'ratios', '', 'results', 'intended', 'part', 'evaluation', 'basis', 'different', 'theoretical', 'treatments', 'problem', '', 'comparative', 'span', 'loading', 'curves', '', 'together', 'supporting', 'evidence', '', 'showed', 'substantial', 'part', 'lift', 'increment', 'produced', 'slipstream', 'due', 'destalling', 'boundarylayercontrol', 'effect', '', 'integrated', 'remaining', 'lift', 'increment', '', 'subtracting', 'destalling', 'lift', '', 'found', 'agree', 'well', 'potential', 'flow', 'theory', '', 'empirical', 'evaluation', 'destalling', 'effects', 'made', 'specific', 'configuration', 'experiment', '']
----------------

## 5. Remove blank space tokens

In [26]:
lent = 0
for i in range(len(newfiles)):
  filedata=newfiles[i]
  tokens_sans_blank_space = remove_blank_space_tokens(filedata)
  newfiles[i] = tokens_sans_blank_space
  lent += len(newfiles[i])
print(len(newfiles))
print(lent)

for i in range(0, 5):
  print('----------')
  print('File : '+str(i+1))
  print('----------')
  print(newfiles[i])
  print('--------------------------------------------------------------------')

1400
127377
----------
File : 1
----------
['experimental', 'investigation', 'aerodynamics', 'wing', 'slipstream', 'experimental', 'study', 'wing', 'propeller', 'slipstream', 'made', 'order', 'determine', 'spanwise', 'distribution', 'lift', 'increase', 'due', 'slipstream', 'different', 'angles', 'attack', 'wing', 'different', 'free', 'stream', 'slipstream', 'velocity', 'ratios', 'results', 'intended', 'part', 'evaluation', 'basis', 'different', 'theoretical', 'treatments', 'problem', 'comparative', 'span', 'loading', 'curves', 'together', 'supporting', 'evidence', 'showed', 'substantial', 'part', 'lift', 'increment', 'produced', 'slipstream', 'due', 'destalling', 'boundarylayercontrol', 'effect', 'integrated', 'remaining', 'lift', 'increment', 'subtracting', 'destalling', 'lift', 'found', 'agree', 'well', 'potential', 'flow', 'theory', 'empirical', 'evaluation', 'destalling', 'effects', 'made', 'specific', 'configuration', 'experiment']
-------------------------------------------------

In [27]:
fileData = newfiles

# QUESTION 2 STARTS HERE


# Q2. Boolean Queries

### 1. Create a unigram inverted index(from scratch; No library allowed) of the dataset obtained from Q1 (after preprocessing).

### 2. Use Python’s pickle module to save and load the unigram inverted index.

In [28]:
def create_unigram_inverted_index(file_dictionary, stopwords_set):
    # initialize unigram inverted index
    unigram_inverted_index = {}
    
    # unigram inverted index
    for doc_ID in range(len(file_dictionary)):
        file = open(file_dictionary[doc_ID], 'r', encoding='utf-8', errors='ignore')
        file_corpus = file.read()
        file.close()
        doc_tokens = preprocess(file_corpus, stopwords_set, 'doc')
        for token in doc_tokens:
            if token in unigram_inverted_index:
                unigram_inverted_index[token][0] += 1               # stores doc_frequency
                unigram_inverted_index[token][1].append(doc_ID)     # appending docs in which that term occurs
            else:
                unigram_inverted_index[token] = []
                unigram_inverted_index[token].append(1)
                unigram_inverted_index[token].append([doc_ID])      # initializing if this token appears for first time
    
    # Saving unigram inverted index using pickle module  
    uii_file = open('unigram_inverted_index_pickle_file', 'wb')
    pickle.dump(unigram_inverted_index, uii_file)
    uii_file.close()

    return unigram_inverted_index

stopwords_set = set(stopwords.words('english'))

list_of_files = getListOfFiles('/content/CSE508_Winter2023_Dataset/')
uni = create_unigram_inverted_index(list_of_files,stopwords_set)

In [29]:
uii_file = open('unigram_inverted_index_pickle_file', 'rb')
unigram_inverted_index = pickle.load(uii_file)
uii_file.close()

In [30]:
print(len(unigram_inverted_index))

8996


### 3. Provide support for the following operations: 
a. T1 AND T2<br>
b. T1 AND NOT T2<br>
c. T1 OR T2<br>
d. T1 OR NOT T2

AND

In [31]:
def xANDy(pos_list_1, pos_list_2):
    result = SortedSet()
    i,j = 0,0
    num_of_operations = 0
    while(i<len(pos_list_1) and j<len(pos_list_2)):
        if pos_list_1[i]==pos_list_2[j]:
            result.add(pos_list_1[i])
            i+=1
            j+=1
        elif pos_list_1[i]<pos_list_2[j]:
            i+=1
        else:
            j+=1
        num_of_operations+=1
    
    return result, num_of_operations

OR

In [32]:
def xORy(pos_list_1, pos_list_2):
    result = SortedSet()
    i,j = 0,0
    num_of_operations = 0
    while(i<len(pos_list_1) and j<len(pos_list_2)):
        if pos_list_1[i]==pos_list_2[j]:
            result.add(pos_list_1[i])
            i+=1
            j+=1
        elif pos_list_1[i]<pos_list_2[j]:
            result.add(pos_list_1[i])
            i+=1
        else:
            result.add(pos_list_2[j])
            j+=1
        num_of_operations+=1
    
    while(i<len(pos_list_1)):
        result.add(pos_list_1[i])
        i+=1
    while(j<len(pos_list_2)):
        result.add(pos_list_2[j])
        j+=1
    
    return result, num_of_operations

AND NOT

In [33]:
def xANDNOTy(pos_list_1, pos_list_2):
    result = SortedSet()
    i,j = 0,0
    num_of_operations = 0
    while(i<len(pos_list_1) and j<len(pos_list_2)):
        if pos_list_1[i]==pos_list_2[j]:
            i+=1
            j+=1
        elif pos_list_1[i]<pos_list_2[j]:
            result.add(pos_list_1[i])
            i+=1
        else:
            j+=1
        num_of_operations+=1
    
    while(i<len(pos_list_1)):
        result.add(pos_list_1[i])
        i+=1
    
    return result, num_of_operations

OR NOT

In [34]:
def xORNOTy(pos_list_1, pos_list_2, u_list):
    num_of_not_operations = 0
    i,j = 0,0
    not_pos_list_2 = SortedSet()
    while(i<len(pos_list_2) and j<len(u_list)):
        if pos_list_2[i]<u_list[j]:
            i+=1
        elif pos_list_2[i]>u_list[j]:
            not_pos_list_2.add(u_list[j])
            j+=1
        else:
            i+=1
            j+=1
        num_of_not_operations+=1
    while(j<len(u_list)):
        not_pos_list_2.add(u_list[j])
        j+=1
    
    result, ops = xORy(pos_list_1, not_pos_list_2)
    return result, num_of_not_operations+ops

### 4. Queries should be generalized i.e., you should provide support for queries like T1 AND T2 OR T3 OR T4 ....

### 5. You are also required to compute the minimum number of comparisons done to execute the query [only where merging is required]

### 6. Input format: <br>
a. The first line contains N denoting the number of queries to execute<br>
b. The next 2N lines contain queries in the following format:<br>
&ensp; i. Input sequence<br>
&ensp; ii. Operations separated by comma

### 7. Output Format:
a. 4N lines consisting of the results in the following format:<br>
&ensp;i. Query X<br>
&ensp;ii. Number of documents retrieved for query X<br>
&ensp;iii. Names of the documents retrieved for query X<br>
&ensp;iv. Number of comparisons required for query X<br>

### 8. Perform preprocessing steps (from Q1) on the input sequence as well.

In [35]:
def search_unigram(query, ops, count):
  # create set of stop words for preprocessing
  stopwords_set = set(stopwords.words('english'))

  # Get List of Files in Dataset
  list_of_files = getListOfFiles('/content/CSE508_Winter2023_Dataset/')

  # create dictionary of file with docID (integer) as key and full_path of file as value
  file_dictionary = create_file_dictionary(list_of_files)

  sanitized_query = preprocess(query, stopwords_set, 'query')

  if(len(sanitized_query) == 0):
    print("unigram invalid query "+str(count)+" -> you had entered stopwords in your query\n")
    return    # will continue with next query

  sanitized_operation_sequence = []

  for op_seq in ops.split(','):
    sanitized_operation_sequence.append(op_seq.upper().strip())

  if(len(sanitized_query)!=len(sanitized_operation_sequence)+1):          
    print('unigram invalid query '+str(count)+' -> number of operations cannot exceed number of tokens in the input query\n')
    return    # will continue with next query 

  print('Query '+str(count)+' : ', end='')

  for i in range(len(sanitized_operation_sequence)):
    print(sanitized_query[i]+" "+sanitized_operation_sequence[i]+" ", end='')
  print(sanitized_query[len(sanitized_operation_sequence)])

  #Loading pre-processed files
  uii_file = open('unigram_inverted_index_pickle_file', 'rb')
  unigram_inverted_index = pickle.load(uii_file)
  uii_file.close()

  u_list = SortedSet()
  for a in range(len(file_dictionary)):
      u_list.add(a)

 

  for token in sanitized_query:
    if(token not in unigram_inverted_index):
      print("unigram invalid query "+str(count)+" -> given token "+token+" is not present in our unigram_inverted_index")
      unigram_inverted_index[token] = []
      unigram_inverted_index[token].append(1)
      unigram_inverted_index[token].append([])
  # return

  ptr1, ptr2 = 0, 0
  result = unigram_inverted_index[sanitized_query[ptr1]][1]
  num_of_operations = 0

  while(ptr2<len(sanitized_operation_sequence)):                          # Processing of the input query X starts here.
              if sanitized_operation_sequence[ptr2]=='AND':
                  res, ops = xANDy(result, unigram_inverted_index[sanitized_query[ptr1+1]][1])
              elif sanitized_operation_sequence[ptr2]=='OR':
                  res, ops = xORy(result, unigram_inverted_index[sanitized_query[ptr1+1]][1])
              elif sanitized_operation_sequence[ptr2]=='AND NOT':
                  res, ops = xANDNOTy(result, unigram_inverted_index[sanitized_query[ptr1+1]][1])
              elif sanitized_operation_sequence[ptr2]=='OR NOT':
                  res, ops = xORNOTy(result, unigram_inverted_index[sanitized_query[ptr1+1]][1], u_list)
              ptr1+=1
              ptr2+=1
              result = res
              num_of_operations += ops

  print('Number of documents retrieved for query '+str(count)+' : '+str(len(result)))
  print('Number of comparisons required for query '+str(count)+' : '+str(num_of_operations))
  print('Names of the documents retrieved for query '+str(count)+' : ', end='')
  # print('Number of documents matched: {}'.format(len(result)))            # Printing Number of documents retrived for query X.
  # print('Number of comparisons required: {}'.format(num_of_operations))   # Printing Number of comparisons required for query X.
  # print('Name of Documents: \n')                                                  # Printing Name of Documents retrived for query X.
  output=[]
  for res_doc in result:
    output.append(file_dictionary[res_doc])
              # print(file_dictionary[res_doc], end=',  ')
  print(output)
  print('\n\n')
  


In [40]:
print(unigram_inverted_index)

{'aerodynamics': [23, [0, 10, 215, 224, 236, 243, 283, 288, 295, 296, 359, 452, 633, 684, 688, 752, 791, 901, 1205, 1270, 1330, 1346, 1379]], 'agree': [32, [0, 13, 71, 164, 176, 232, 288, 317, 329, 362, 382, 416, 647, 665, 687, 727, 829, 868, 886, 902, 922, 949, 958, 995, 1117, 1184, 1190, 1198, 1203, 1268, 1302, 1366]], 'angles': [94, [0, 24, 57, 63, 68, 105, 163, 172, 188, 192, 196, 198, 211, 224, 228, 281, 286, 290, 314, 353, 359, 362, 372, 419, 422, 433, 439, 442, 463, 483, 491, 504, 519, 563, 564, 566, 609, 625, 631, 649, 672, 673, 687, 707, 708, 710, 711, 712, 714, 716, 735, 785, 787, 791, 800, 813, 814, 906, 923, 926, 935, 946, 992, 998, 1076, 1094, 1103, 1113, 1114, 1146, 1180, 1192, 1212, 1217, 1228, 1230, 1249, 1258, 1261, 1276, 1284, 1303, 1305, 1306, 1308, 1319, 1323, 1338, 1340, 1344, 1346, 1348, 1349, 1380]], 'attack': [105, [0, 26, 31, 47, 55, 56, 57, 68, 69, 121, 123, 188, 192, 196, 224, 230, 231, 233, 247, 311, 353, 359, 362, 372, 419, 432, 433, 438, 439, 440, 442, 468

In [37]:
N = int(input('enter number of queries: '))
inputs = []
input_ops = []
count = 1

for q in range(N):
  inp_sent = input('enter query '+str(q+1)+': ')
  inputs.append(inp_sent)

  inp_ops_seq = input('enter operation sequence '+str(q+1)+': ')
  input_ops.append(inp_ops_seq)

print('\n')

for query, ops in zip(inputs, input_ops):
  search_unigram(query, ops, count)
  count = count+1

enter number of queries: 6
enter query 1: downstream locations along the jet
enter operation sequence 1: and, or, or
enter query 2: experimental
enter operation sequence 2: or
enter query 3: experimental investigation
enter operation sequence 3: and not
enter query 4: a is the #$%@@
enter operation sequence 4: or not
enter query 5: experimental invention experiment
enter operation sequence 5: and, or
enter query 6: taasir experimental
enter operation sequence 6: or


Query 1 : downstream AND locations OR along OR jet
Number of documents retrieved for query 1 : 216
Number of comparisons required for query 1 : 395
Names of the documents retrieved for query 1 : ['/content/CSE508_Winter2023_Dataset/cranfield0007', '/content/CSE508_Winter2023_Dataset/cranfield0009', '/content/CSE508_Winter2023_Dataset/cranfield0023', '/content/CSE508_Winter2023_Dataset/cranfield0034', '/content/CSE508_Winter2023_Dataset/cranfield0040', '/content/CSE508_Winter2023_Dataset/cranfield0056', '/content/CSE508_Win